In [148]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

In [ ]:
import numpy as np


def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [151]:
with open('result/2_iter_user_based_interesting_users.pickle', 'rb') as f:
    user_based_prediction = pickle.load(f)
    
with open('result/2_iter_item_based_interesting_users.pickle', 'rb') as f:
    item_based_prediction = pickle.load(f)
    
with open('result/2_iter_most_popular_full.pickle', 'rb') as f:
    most_popular_prediction = pickle.load(f)

with open('users_subsample.pickle', 'rb') as input:
    users_ids = pickle.load(input)
        
with open('result/2_iter_full_actual_purchases.pickle', 'rb') as f:
    actual_purchases = pickle.load(f)

In [8]:
# Наши 2к юзеров
with open('result/users_subsample.pickle', 'rb') as f:
    users = pickle.load(f)
    
products = pd.read_csv('data/products.csv')

In [4]:
#Первая итерация

with open('result/1_iter_ensenble_predictions_models.pickle', 'rb') as f:
    iter1_ensemble_model = pickle.load(f)
    
with open('result/1_iter_ensenble_predictions.pickle', 'rb') as f:
    iter1_ensemble = pickle.load(f)
    
with open('result/1_iter_full_actual_purchases.pickle', 'rb') as f:
    iter1_actual = pickle.load(f)
    
with open('result/1_iter_item_based_interesting_users.pickle', 'rb') as f:
    iter1_item_based = pickle.load(f)
    
with open('result/1_iter_most_popular_full.pickle', 'rb') as f:
    iter1_most_popular = pickle.load(f)
    
with open('result/1_iter_user_based_interesting_users.pickle', 'rb') as f:
    iter1_user_based = pickle.load(f)
    

In [5]:
#Вторая итерация

with open('result/2_iter_ensenble_predictions_models.pickle', 'rb') as f:
    iter2_ensemble_model = pickle.load(f)
    
with open('result/2_iter_ensenble_predictions.pickle', 'rb') as f:
    iter2_ensemble = pickle.load(f)
    
with open('result/2_iter_full_actual_purchases.pickle', 'rb') as f:
    iter2_actual = pickle.load(f)
    
with open('result/2_iter_item_based_interesting_users.pickle', 'rb') as f:
    iter2_item_based = pickle.load(f)
    
with open('result/2_iter_most_popular_full.pickle', 'rb') as f:
    iter2_most_popular = pickle.load(f)
    
with open('result/2_iter_user_based_interesting_users.pickle', 'rb') as f:
    iter2_user_based = pickle.load(f)

<h3> Возьмем троих юзеров <br>
1: Частопокупающий топовые товары (Бананы и органическую фигню))) <br>
2: Юзер-наоборот <br>
3: И Рандомный</h3>

In [130]:

keys = [user for user in users]

user_top = keys[1]
user_ne_top = keys[900]
user_random = keys[1500]
print(f'1)Юзер топ: {user_top}\n\
2)Юзер не топ: {user_ne_top}\n\
3)Рандом: {user_random}')

1)Юзер топ: 199095
2)Юзер не топ: 138451
3)Рандом: 128653


<h2>Первая итерация</h2>

<h3>User top</h3>

In [131]:
df_user_top = pd.DataFrame([])


# Мост попьюлар
df_user_top['most_popular'] = products.loc[products[' product_id'].isin(iter1_most_popular[user_top])]['product_name'].values

# Item based
df_user_top['item_based'] = products.loc[products[' product_id'].isin(iter1_item_based[user_top])]['product_name'].values

# User based
df_user_top['user_based'] = products.loc[products[' product_id'].isin(iter1_user_based[user_top][:10])]['product_name'].values

# Ensemble
df_user_top['ensemble'] = products.loc[products[' product_id'].isin(iter1_ensemble[user_top][:10])]['product_name'].values

# Actual
df_user_top['actual'] = products.loc[products[' product_id'].isin(iter1_actual[user_top][:10])]['product_name'].values

df_user_top

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Clementines,Packaged Grape Tomatoes,Clementines,Seedless Red Grapes
1,Strawberries,Hass Avocados,Baby Cucumbers,Hass Avocados,Clementines
2,Organic Strawberries,Bag of Organic Bananas,Hass Avocados,Bag of Organic Bananas,Organic Animal Crackers
3,Organic Baby Spinach,Strawberries,Bag of Organic Bananas,Strawberries,Hass Avocados
4,Banana,Blackberries,Strawberries,Blackberries,Bag of Organic Bananas
5,Limes,Braeburn Apples,Blackberries,Braeburn Apples,Strawberries
6,Organic Whole Milk,Pears,Rainbow Bell Peppers,Pears,Organic Strawberries
7,Organic Hass Avocado,Organic Blueberries,Seedless Cucumbers,Organic Blueberries,Blackberries
8,Large Lemon,Raspberries,Organic Blueberries,Raspberries,Organic Blueberries
9,Organic Avocado,"8\"" Vegan Chocolate Cake",Raspberries,"8\"" Vegan Chocolate Cake",Raspberries


In [167]:
one = list(products.loc[products[' product_id'].isin(iter1_item_based[user_top])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter1_actual[user_top][:10])][' product_id'].values)
metric = apk(one, two)

In [173]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_top].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.4258333333333333


<h3>User ne top</h3>

In [114]:
df_user_ne_top = pd.DataFrame([])

# Мост попьюлар
df_user_ne_top['most_popular'] = products.loc[products[' product_id'].isin(iter1_most_popular[user_ne_top])]['product_name'].values

# Item based
df_user_ne_top['item_based'] = products.loc[products[' product_id'].isin(iter1_item_based[user_ne_top])]['product_name'].values

# User based
df_user_ne_top['user_based'] = products.loc[products[' product_id'].isin(iter1_user_based[user_ne_top][:10])]['product_name'].values

# Ensemble
df_user_ne_top['ensemble'] = products.loc[products[' product_id'].isin(iter1_ensemble[user_ne_top][:10])]['product_name'].values

# Actual
df_user_ne_top['actual'] = products.loc[products[' product_id'].isin(iter1_actual[user_ne_top][:10])]['product_name'].values

df_user_ne_top

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Non-Fat Blueberry on the Bottom Greek Yogurt,Organic 2% Reduced Fat Milk,Organic 2% Reduced Fat Milk,Organic Southwestern Burrito
1,Strawberries,Peach-Pear Sparkling Water,Fridge Pack Cola,Fridge Pack Cola,Non-Fat Blueberry on the Bottom Greek Yogurt
2,Organic Strawberries,"Sparkling Water, Natural Mango Essenced",Kiwi Sandia Sparkling Water,Kiwi Sandia Sparkling Water,Pork Carnitas Burrito
3,Organic Baby Spinach,Kiwi Sandia Sparkling Water,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% All Natural Greek Strained Yogurt wit...,Pineapple Strawberry Sparkling Water
4,Banana,Pineapple Strawberry Sparkling Water,Curate Cherry Lime Sparkling Water,Curate Cherry Lime Sparkling Water,Gluten Free Dark Chocolate Cocoa Breakfast Bars
5,Limes,0% Greek Yogurt Black Cherry on the Bottom,Passionfruit Sparkling Water,Passionfruit Sparkling Water,Peach on the Bottom Nonfat Greek Yogurt
6,Organic Whole Milk,Hummus Roasted Pepper,Lemon Sparkling Water,Lemon Sparkling Water,Cocoa Roasted Almonds
7,Organic Hass Avocado,Cantaloupe Grapefruit Sparkling Water,Grapefruit Sparkling Water,Grapefruit Sparkling Water,Pitted Organic Kalamata Olives To Go!
8,Large Lemon,Cocoa Roasted Almonds,Organic Half & Half,Organic Half & Half,Non-Fat Greek Yogurt With Strawberries on the ...
9,Organic Avocado,Curate Pomme Baya-Apple Berry Sparkling Water,Orange Sparkling Water,Orange Sparkling Water,Organic Gluten Free Non-Dairy Beans & Rice Bur...


In [186]:
one = list(products.loc[products[' product_id'].isin(iter1_item_based[user_ne_top])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter1_actual[user_ne_top][:10])][' product_id'].values)
metric = apk(one, two)

In [187]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_ne_top].upper()}\nМетрика = {metric}')

Лучшая моделька: USER BASED
Метрика = 0.14285714285714285


<h3>Random user</h3>

In [132]:
df_user_random = pd.DataFrame([])

# Мост попьюлар
df_user_random['most_popular'] = products.loc[products[' product_id'].isin(iter1_most_popular[user_random])]['product_name'].values

# Item based
df_user_random['item_based'] = products.loc[products[' product_id'].isin(iter1_item_based[user_random])]['product_name'].values

# User based
df_user_random['user_based'] = products.loc[products[' product_id'].isin(iter1_user_based[user_random][:10])]['product_name'].values

# Ensemble
df_user_random['ensemble'] = products.loc[products[' product_id'].isin(iter1_ensemble[user_random][:10])]['product_name'].values

# Actual
df_user_random['actual'] = products.loc[products[' product_id'].isin(iter1_actual[user_random][:10])]['product_name'].values

df_user_random

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Organic Spinach Bunch,Organic Baby Spinach,Organic Spinach Bunch,85% Lean Ground Beef
1,Strawberries,Organic Baby Spinach,Organic D'Anjou Pears,Organic Baby Spinach,Green Beans
2,Organic Strawberries,Organic Large Green Asparagus,Banana,Organic Large Green Asparagus,Organic Grade A Free Range Large Brown Eggs
3,Organic Baby Spinach,Organic Broccoli Garlic Poppers,Fresh Cauliflower,Organic Broccoli Garlic Poppers,Organic Baby Spinach
4,Banana,Organic Egg Whites,Organic Kiwi,Organic Egg Whites,Tofu Scramble
5,Limes,Organic Baby Broccoli,Organic Grape Tomatoes,Organic Baby Broccoli,Extra Virgin Olive Oil
6,Organic Whole Milk,Organic Baby Kale,Organic Zucchini,Organic Baby Kale,Roasted Chicken Breast
7,Organic Hass Avocado,Organic Spring Mix,Asparagus,Organic Spring Mix,Organic Gala Apples
8,Large Lemon,Organic Mixed Baby Kale Salad,Organic Avocado,Organic Mixed Baby Kale Salad,Roasted Almond Butter
9,Organic Avocado,Roasted Turkey Breast,Cucumber Kirby,Roasted Turkey Breast,Ground Buffalo


In [184]:
one = list(products.loc[products[' product_id'].isin(iter1_item_based[user_random])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter1_actual[user_random][:10])][' product_id'].values)
metric = apk(one, two)

In [185]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_random].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.025


<hr>

<h2>Вторая итерация</h2>

<h3>User top</h3>

In [133]:
df_user_top = pd.DataFrame([])


# Мост попьюлар
df_user_top['most_popular'] = products.loc[products[' product_id'].isin(iter2_most_popular[user_top])]['product_name'].values

# Item based
df_user_top['item_based'] = products.loc[products[' product_id'].isin(iter2_item_based[user_top])]['product_name'].values

# User based
df_user_top['user_based'] = products.loc[products[' product_id'].isin(iter2_user_based[user_top][:10])]['product_name'].values

# Ensemble
df_user_top['ensemble'] = products.loc[products[' product_id'].isin(iter2_ensemble[user_top][:10])]['product_name'].values

# Actual
df_user_top['actual'] = products.loc[products[' product_id'].isin(iter2_actual[user_top][:10])]['product_name'].values

df_user_top

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Hass Avocados,Packaged Grape Tomatoes,Hass Avocados,Seedless Red Grapes
1,Strawberries,Bag of Organic Bananas,Baby Cucumbers,Bag of Organic Bananas,Clementines
2,Organic Strawberries,Strawberries,Hass Avocados,Strawberries,Organic Animal Crackers
3,Organic Baby Spinach,Blackberries,Bag of Organic Bananas,Blackberries,Hass Avocados
4,Banana,Braeburn Apples,Strawberries,Braeburn Apples,Bag of Organic Bananas
5,Limes,Sleepy Hollow Pinot Noir,Blackberries,Sleepy Hollow Pinot Noir,Organic Strawberries
6,Organic Whole Milk,Pears,Rainbow Bell Peppers,Pears,Blackberries
7,Organic Hass Avocado,Organic Blueberries,Seedless Cucumbers,Organic Blueberries,Braeburn Apples
8,Large Lemon,Raspberries,Organic Blueberries,Raspberries,Organic Blueberries
9,Organic Avocado,"8\"" Vegan Chocolate Cake",Raspberries,"8\"" Vegan Chocolate Cake","8\"" Vegan Chocolate Cake"


In [178]:
one = list(products.loc[products[' product_id'].isin(iter2_item_based[user_top])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter2_actual[user_top][:10])][' product_id'].values)
metric = apk(one, two)

In [179]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_top].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.27341269841269844


<h3>User ne top</h3>

In [134]:
df_user_ne_top = pd.DataFrame([])

# Мост попьюлар
df_user_ne_top['most_popular'] = products.loc[products[' product_id'].isin(iter2_most_popular[user_ne_top])]['product_name'].values

# Item based
df_user_ne_top['item_based'] = products.loc[products[' product_id'].isin(iter2_item_based[user_ne_top])]['product_name'].values

# User based
df_user_ne_top['user_based'] = products.loc[products[' product_id'].isin(iter2_user_based[user_ne_top][:10])]['product_name'].values

# Ensemble
df_user_ne_top['ensemble'] = products.loc[products[' product_id'].isin(iter2_ensemble[user_ne_top][:10])]['product_name'].values

# Actual
df_user_ne_top['actual'] = products.loc[products[' product_id'].isin(iter2_actual[user_ne_top][:10])]['product_name'].values

df_user_ne_top

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Peach-Pear Sparkling Water,Organic 2% Reduced Fat Milk,Organic 2% Reduced Fat Milk,Organic Southwestern Burrito
1,Strawberries,Mango Sparkling Water,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% Lowfat Greek Strained Yogurt With Blu...,Non-Fat Blueberry on the Bottom Greek Yogurt
2,Organic Strawberries,Cran-Raspberry Flavored Sparkling Water,Fridge Pack Cola,Fridge Pack Cola,Pork Carnitas Burrito
3,Organic Baby Spinach,"Sparkling Water, Natural Mango Essenced",Total 2% All Natural Greek Strained Yogurt wit...,Total 2% All Natural Greek Strained Yogurt wit...,Pineapple Strawberry Sparkling Water
4,Banana,Kiwi Sandia Sparkling Water,2% Reduced Fat Milk,2% Reduced Fat Milk,Gluten Free Dark Chocolate Cocoa Breakfast Bars
5,Limes,Pineapple Strawberry Sparkling Water,Curate Cherry Lime Sparkling Water,Curate Cherry Lime Sparkling Water,Peach on the Bottom Nonfat Greek Yogurt
6,Organic Whole Milk,Hummus Roasted Pepper,Passionfruit Sparkling Water,Passionfruit Sparkling Water,Cocoa Roasted Almonds
7,Organic Hass Avocado,Cantaloupe Grapefruit Sparkling Water,Grapefruit Sparkling Water,Grapefruit Sparkling Water,Pitted Organic Kalamata Olives To Go!
8,Large Lemon,Cocoa Roasted Almonds,Organic Half & Half,Organic Half & Half,Non-Fat Greek Yogurt With Strawberries on the ...
9,Organic Avocado,Curate Pomme Baya-Apple Berry Sparkling Water,Orange Sparkling Water,Orange Sparkling Water,Organic Gluten Free Non-Dairy Beans & Rice Bur...


In [180]:
one = list(products.loc[products[' product_id'].isin(iter2_item_based[user_ne_top])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter2_actual[user_ne_top][:10])][' product_id'].values)
metric = apk(one, two)

In [182]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_ne_top].upper()}\nМетрика = {metric}')

Лучшая моделька: USER BASED
Метрика = 0.05357142857142857


<h3>Random User</h3>

In [135]:
df_user_random = pd.DataFrame([])

# Мост попьюлар
df_user_random['most_popular'] = products.loc[products[' product_id'].isin(iter2_most_popular[user_random])]['product_name'].values

# Item based
df_user_random['item_based'] = products.loc[products[' product_id'].isin(iter2_item_based[user_random])]['product_name'].values

# User based
df_user_random['user_based'] = products.loc[products[' product_id'].isin(iter2_user_based[user_random][:10])]['product_name'].values

# Ensemble
df_user_random['ensemble'] = products.loc[products[' product_id'].isin(iter2_ensemble[user_random][:10])]['product_name'].values

# Actual
df_user_random['actual'] = products.loc[products[' product_id'].isin(iter2_actual[user_random][:10])]['product_name'].values

df_user_random

,most_popular,item_based,user_based,ensemble,actual
0,Bag of Organic Bananas,Bok Choy,Organic Lemon,Bok Choy,85% Lean Ground Beef
1,Strawberries,Organic Spinach Bunch,Carrots,Organic Spinach Bunch,Green Beans
2,Organic Strawberries,Organic Large Green Asparagus,Organic Baby Arugula,Organic Large Green Asparagus,Organic Grade A Free Range Large Brown Eggs
3,Organic Baby Spinach,Organic Broccoli Garlic Poppers,Organic Baby Spinach,Organic Broccoli Garlic Poppers,Organic Baby Spinach
4,Banana,Organic Egg Whites,Red Peppers,Organic Egg Whites,Tofu Scramble
5,Limes,Organic Baby Broccoli,Banana,Organic Baby Broccoli,Extra Virgin Olive Oil
6,Organic Whole Milk,Organic Baby Kale,Fresh Cauliflower,Organic Baby Kale,Roasted Chicken Breast
7,Organic Hass Avocado,Organic Spring Mix,Organic Grape Tomatoes,Organic Spring Mix,Organic Gala Apples
8,Large Lemon,Organic Mixed Baby Kale Salad,Organic Zucchini,Organic Mixed Baby Kale Salad,Roasted Almond Butter
9,Organic Avocado,Roasted Turkey Breast,Cucumber Kirby,Roasted Turkey Breast,Ground Buffalo


In [188]:
one = list(products.loc[products[' product_id'].isin(iter2_item_based[user_random])][' product_id'].values)
two = list(products.loc[products[' product_id'].isin(iter2_actual[user_random][:10])][' product_id'].values)
metric = apk(one, two)

In [189]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_random].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.0
